### Use TCGA expression data to train RB1 signature

In [183]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

#### 1. TCGA expression data and RB1 mutation data preprocessing 

In [49]:
mRNA_file = "./data/TCGA/mRNA.tsv"
mutation_file = "./data/TCGA/RB1_maf.tsv"
clinical_file = "./data/TCGA/clinical.tsv"

In [23]:
mRNA_df = pd.read_csv(mRNA_file, delimiter="\t")
# dropna
mRNA_df.dropna(inplace=True)
# transpose
mRNA_df_transposed = mRNA_df.set_index("gene_id").transpose()
# take log
mRNA_df_transposed_log = mRNA_df_transposed.applymap(lambda x:np.log(x+1))
# save preprocessed mRNA (dropna, transpose, log)
mRNA_df_transposed_log.index.name = "sample_id"
mRNA_df_transposed_log.to_csv("./data/TCGA/mRNA_transpose_log.csv")

In [27]:
# get just prostate tissue
def get_label(barcode, dic):
    patient_id = barcode[:12]
    if patient_id in dic:
        return dic[patient_id]
    else:
        return None

mRNA_df = pd.read_csv("./data/TCGA/mRNA_transpose_log.csv")
clinical_df = pd.read_csv(clinical_file, delimiter="\t", 
                          encoding="ISO-8859-1", low_memory=False)
tissue_dict = dict(zip(clinical_df.bcr_patient_barcode, clinical_df.acronym))
label_df = pd.DataFrame(index=mRNA_df.sample_id)
label_df["tissue"] = list(mRNA_df.sample_id.apply(lambda x: get_label(x, tissue_dict)))
prostate_samples = list(label_df[label_df.tissue == "PRAD"].index)
prostate_mRNA_df = mRNA_df[mRNA_df.sample_id.isin(prostate_samples)]
prostate_mRNA_df.to_csv("./data/TCGA/mRNA_transpose_log_prostate.csv")

In [139]:
# get RB1 mutation status
def is_RB1_mutated(variant_classification):
    if variant_classification in ["Silent", "3'UTR", "Intron"]:
        return 0
    else:
        return 1
    
def add_to_label(barcode):
    if barcode[:15] not in RB1_dict.keys():
        return 0
    else:
        return RB1_dict[barcode[:15]]
    
    
rb_df = pd.read_csv(mutation_file, delimiter="\t")
rb_df["RB1_mutated"] = rb_df.Variant_Classification.apply(is_RB1_mutated)
RB1_dict = dict(zip([i[:15] for i in rb_df.Tumor_Sample_Barcode], 
                    rb_df.RB1_mutated))

assert(list(label_df.index) == list(mRNA_df.sample_id))
label_df["RB_mutated"] = list(map(add_to_label, list(label_df.index)))
label_df.to_csv("./data/TCGA/RB_labels.csv")

#### 2. Training Signature with Logistic Regression (with L1 regularization)

In [175]:
y = label_df.RB_mutated.as_matrix()
X = mRNA_df.set_index("sample_id").as_matrix()

In [207]:
# Five fold stratified cross validation
# high regularization C=0.01
# balance class weight with label frequency

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)
result_df = pd.DataFrame()
fold_num = 0
genes_used = []
for train_idx, test_idx in skf.split(X, y):
    fold_num += 1
    print(fold_num)
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    lr = LogisticRegression(penalty="l1", C=0.002,
                           class_weight="balanced", n_jobs=16)
    lr.fit(X_train, y_train)
    n_features_used = np.count_nonzero(lr.coef_)
    genes_used.append(lr.coef_)
    y_pred = lr.predict(X_test)
    test_auc = roc_auc_score(y_test, y_pred)
    train_auc = roc_auc_score(y_train, lr.predict(X_train))
    result_df.loc["train AUC", fold_num] = train_auc
    result_df.loc["test AUC", fold_num] = test_auc
    result_df.loc["num features", fold_num] = n_features_used

1
2
3
4
5


In [208]:
result_df

,1,2,3,4,5
train AUC,0.835235,0.855797,0.837267,0.841670,0.836575
test AUC,0.777665,0.629423,0.688685,0.719555,0.677811
num features,48.000000,45.000000,49.000000,49.000000,27.000000


In [212]:
print(result_df.mean(axis=1))

train AUC        0.841309
test AUC         0.698628
num features    43.600000
dtype: float64


In [213]:
print(result_df.std(axis=1))

train AUC       0.008451
test AUC        0.054786
num features    9.423375
dtype: float64
